In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

In [2]:
years = list(range(2023, 2013, -1))
all_matches = []

In [ ]:
for year in years:
    print(f"{year-1}-{year}")

In [3]:
standings_url = "https://fbref.com/en/comps/9/2022-2023/2022-2023-Premier-League-Stats"

In [4]:
data = requests.get(standings_url)
soup = BeautifulSoup(data.text)

In [ ]:
data.text

In [ ]:
soup

In [5]:
standings_table = soup.select('table.stats_table')[0]
links = [l.get("href") for l in standings_table.find_all('a')]
links = [l for l in links if '/squads/' in l]
team_urls = [f"https://fbref.com{l}" for l in links]

In [6]:
first_3_clubs_urls = team_urls[:3]
first_3_clubs_urls

['https://fbref.com/en/squads/b8fd03ef/2022-2023/Manchester-City-Stats',
 'https://fbref.com/en/squads/18bb7c10/2022-2023/Arsenal-Stats',
 'https://fbref.com/en/squads/19538871/2022-2023/Manchester-United-Stats']

In [ ]:
# soup.find('div', class_='prevnext').find('a')['href']

In [ ]:
# data = requests.get(team_url)
# matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

In [7]:
for team_url in first_3_clubs_urls:
    all_matches.clear()
    team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
    data = requests.get(team_url)
    soup = BeautifulSoup(data.text)
    for year in years:
        # Scores & Fixtures
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        # Shooting
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "SoT", "PKatt", "Opponent"]], on=["Date", "Opponent"], how='inner')
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        team_data["Season"] = f"{year-1}-{year}"
        team_data["Team"] = team_name
        team_data["url"] = f"https://fbref.com{links[0]}"
        all_matches.append(team_data)
        time.sleep(1)

        previous_season = soup.find('div', class_='prevnext').find('a')['href']
        # previous_season = soup.select("a.prev")[0].get("href")
        standings_url = f"https://fbref.com{previous_season}"
        data = requests.get(standings_url)
    match_df = pd.concat(all_matches)
    match_df.columns = [c.lower() for c in match_df.columns]
    match_df.to_csv(f"{team_name.lower()}.csv", index=False)


C:\Users\Admin\AppData\Local\Temp\ipykernel_9564\1280025251.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
C:\Users\Admin\AppData\Local\Temp\ipykernel_9564\1280025251.py:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]
C:\Users\Admin\AppData\Local\Temp\ipykernel_9564\1280025251.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
C:\Users\Admin\AppData\Local\Temp\ipykernel_9564\1280025251.py:14: FutureWarning: Passing literal html t